In [ ]:
import os; os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
from tensorflow import keras

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    '03_dl/02_cnn/data/cats_dogs/',
    batch_size=16,
    image_size=(256, 256),
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='training')

val_ds = keras.utils.image_dataset_from_directory(
    '03_dl/02_cnn/data/cats_dogs/',
    batch_size=16,
    image_size=(256, 256),
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='validation')

In [ ]:
model = keras.Sequential()
model.add(keras.layers.InputLayer(input_shape=(256, 256, 3)))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1))

model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=[
        keras.metrics.BinaryAccuracy(),
        keras.metrics.Precision(),
        keras.metrics.Recall()
    ]
)

Andiamo ad inserire tre diversi callback:

1. un `ModelCheckpoint`, che salva i risultati intermedi nel file da noi specificati;
2. un `EarlyStopping`, che interrompe l'addestramento qualora i risultati ottenuti non migliorino;
3. un `TensorBoard`, che fa in modo che i risultati siano mostrati mediante TensorBoard.

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='checkpoints/checkpoints',
        save_weights_only=True,
        monitor='val_binary_accuracy',
        save_best_only=True),
    keras.callbacks.EarlyStopping(
        monitor='val_binary_accuracy',
        min_delta=0.1,
        patience=3,
        restore_best_weights=True),
    keras.callbacks.TensorBoard(log_dir='logs')
]

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)

In [ ]:
model.save('results/saved_cat_dogs.h5')